    给你一个整数数组 nums ，按要求返回一个新数组 counts 。

    数组 counts 有该性质： counts[i] 的值是  nums[i] 右侧小于 nums[i] 的元素的数量。

示例 1：

    输入：nums = [5,2,6,1]
    输出：[2,1,1,0] 
    解释：
    5 的右侧有 2 个更小的元素 (2 和 1)
    2 的右侧仅有 1 个更小的元素 (1)
    6 的右侧有 1 个更小的元素 (1)
    1 的右侧有 0 个更小的元素

示例 2：

    输入：nums = [-1]
    输出：[0]

示例 3：

    输入：nums = [-1,-1]
    输出：[0,0]
 
提示：

    1 <= nums.length <= 105
    -104 <= nums[i] <= 104

作者：LeetCode

链接：https://leetcode.cn/leetbook/read/top-interview-questions-hard/xdoube/

来源：力扣（LeetCode）

著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

解题思路：

    1、有序数组+二分搜索，维护sl有序数组，从右到左·

In [ ]:
'''BIT：Binary Indexed Tree 树状数组'''
class BIT:
    def __init__(self, n):
        self.tree = [0] * (n+1)     # 比原数组多1个元素
    
    def lowbit(self, x):
        return x & (-x)
    
    def add(self, i, delta):        # 单点更新：执行+delta
        i += 1                      # 原数组下标转换到树状数组下标
        while i<len(self.tree):
            self.tree[i] += delta
            i += self.lowbit(i)

    def query(self, i):             # 前缀和查询
        i += 1                      # 原数组下标转换到树状数组下标
        summ = 0
        while i>0:
            summ += self.tree[i]
            i -= self.lowbit(i)
        return summ

In [10]:
from typing import List
from sortedcontainers import SortedList
class Solution:
    def countSmaller0(self, nums: List[int]) -> List[int]:
        n = len(nums)
        res = [0] * n
        sl = SortedList()

        def biserch(num, low, high):
            left = low
            right = high
            while left < right:
                mid = (left + right) // 2
                if sl[mid] < num:
                    left = mid + 1
                else:
                    right = mid
            return left

        for i in range(n-1, -1, -1):
            cnt = biserch(nums[i], 0, len(sl)-1)      # 二分查找，找到最大的小于nums[i]的数，也就是找到num[i]右侧小于它的个数。
            res[i] = cnt                              # 根据二分查找返回的下标值便是，右侧小于的个数
            sl.add(nums[i])

        return res
    
    def countSmaller1(self, nums: List[int]) -> List[int]:
        n = len(nums)
        res = [0] * n

        def mergesort(nums, low, high):
            if low >= high:
                return None
            mid = (low + high) // 2
            mergesort(nums, low, mid)
            mergesort(nums, mid+1, high)

            # nums[low: mid] 和 nums[mid+1: high] 已经排序好
            tmp = []
            left, right = low, mid+1
            while left <= mid and right <= high:    # 归并操作
                if nums[left][0] <= nums[right][0]:
                    tmp.append(nums[left])
                    res[nums[left][1]] += right - (mid + 1)
                    left += 1
                else:
                    tmp.append(nums[right])
                    right += 1
            
            while left <= mid:  # 左边有剩余
                tmp.append(nums[left])
                res[nums[left][1]] += right - (mid + 1)
                left += 1

            while right <= high:    # 右边有剩余
                tmp.append(nums[right])
                right += 1

            nums[low:high+1] = tmp

        # 带有原始下标的数组
        nums = [(num, idx) for idx, num in enumerate(nums)]

        # 归并排序
        mergesort(nums, 0, n-1)

        return res
    
    def countSmaller2(self, nums: List[int]) -> List[int]:
        n = len(nums)
        
        # 离散化：绝对数值转秩次【rank从0开始】
        uniques = sorted(set(nums))
        rank_map = {v:i for i,v in enumerate(uniques)}
        
        # 构建树状数组
        tree = BIT(len(uniques))
        
        # 从右往左查询
        res = [0] * n
        for i in range(n-1, -1, -1):
            rank = rank_map[nums[i]]    # 当前值的排名
            tree.add(rank, 1)           # 单点更新+1
            res[i] = tree.query(rank-1)    # 查询 当前排名之前 的元素有多少个

        return res

In [11]:
import time
def main():
    start = time.time() * 1000
    nums = [26,78,27,100,33,67,90,23,66,5,38,7,35,23,52,22,83,51,98,69,81,32,78,28,94,13,2,97,3,76,99,51,9,21,84,66,65,36,100,41]
    s = Solution()
    answer = s.countSmaller1(nums)
    print(answer)
    end = time.time() * 1000
    print(f"spend time {end - start}/milliseconds")
    return None

In [12]:
if __name__ == "__main__":
    main()

[5, 13, 5, 19, 7, 12, 16, 5, 11, 2, 8, 2, 7, 5, 10, 5, 15, 9, 18, 12, 7, 2, 7, 2, 8, 1, 0, 8, 0, 7, 4, 2, 0, 0, 4, 1, 1, 0, 1, 0]
spend time 0.998046875/milliseconds
